# PowerShell Classes

PowerShell class support was rolled out in PowerShell v5 to enable authoring DSC resources easier. But PowerShell classes bring much more to the table.

## Function vs Class

Writing functions is a procedural way of doing things, where we break down a task into a set of procedures or functions. Functions are a good candidate for an interactive experience for an end-user and focus more on it.

Whereas using Class is an object-oriented approach is to break down a programming task into objects that expose behavior (methods) and data (members or attributes) using interfaces. Classes are good for modeling complicated systems and writing more maintainable code.

Bear this in mind when you want to leverage PowerShell classes in your day to day work.

## Class

Class are blueprints for the objects. In the class you define what data to store (properties) and behavior (methods) that manipulates the data stored.

In [49]:
Class Computer {} # Empty class

In [50]:
[Computer]::new() # Create a new class

Computer


## Properties

Let's add some Computer class properties which are present on an actual Computer object in real world.

In [51]:
Class Computer {
    $hostName
    $ipAddress
}

In [52]:
$Computer = [Computer]::new() # Create an instance 
$Computer.hostName = "testvm1" # provide the property values after creating
$Computer.ipAddress = "10.10.10.10"

In [53]:
$Computer


hostName ipAddress
-------- ---------
testvm1  10.10.10.10



> Trick - Cast initialization (PowerShell syntactic sugar)

Create an instance and provide all the property values in one step.

In [54]:
$Computer = [Computer] @{ "hostName" = "testvm1"; "ipAddress" = "10.10.10.10" }
$Computer


hostName ipAddress
-------- ---------
testvm1  10.10.10.10



### TypeCasting Properties

We can place a type on our Class properties to make them strongly typed and take care of some validation. 

In [55]:
Class Computer {
    [String] $hostName
    [Net.IpAddress] $ipAddress
}

In [56]:
$Computer = [Computer] @{ "hostName" = "testvm1"; "ipAddress" = "10.10.10.10" }
$Computer


hostName ipAddress
-------- ---------
testvm1  10.10.10.10



In [57]:
$Computer = [Computer] @{ "hostName" = "testvm1"; "ipAddress" = "10.256.10.10" } # See how typecasting takes care of some validation logic

InvalidArgument: Cannot create object of type "Computer". Cannot convert value "10.256.10.10" to type "System.Net.IPAddress". Error: "An invalid IP address was specified."


### Validate Attributes for Properties

In PowerShell function we often use Validate attributes, we can do the same here. 

In [58]:
Class Computer {
    [ValidateLength(5, 15)] # hostname in AD can't be more than 15 chars
    [String] $hostName
    
    [Net.IpAddress] $ipAddress
}

In [59]:
[Computer] @{ "hostName" = "test-vm1-thisisalongname"; "ipAddress" = "10.10.10.10" }

InvalidArgument: Cannot create object of type "Computer". The character length of the 24 argument is too long. Shorten the character length of the argument so it is fewer than or equal to "15" characters, and then try the command again.


## Methods

Methods define the behavior of the object of the class, it can leverage the properties on the object instance.

Let's add a method which checks if the machine is alive for a computer. 

In [60]:
Class Computer {
    [ValidateLength(5, 15)] # hostname in AD can't be more than 15 chars
    [String] $hostName
    
    [Net.IpAddress] $ipAddress
    
    [bool] IsAlive() {
        return Test-Connection -ComputerName $this.ipAddress -Count 2 -Quiet
    }
}

In [61]:
$Computer = [Computer] @{ "hostName" = "localhost"; "ipAddress" = "127.0.0.1" }

In [62]:
$Computer.IsAlive()

True


## Constructors

Up until now we have been specifying all the properties ourselves while creating an instance of the class.
However, you can add a constructor (similar to method) to place an initialization logic for the instance of your class.

For our computer class, let's take the FQDN as the input for the initialization.

In [63]:
Class Computer {
    [ValidateLength(5, 15)] # hostname in AD can't be more than 15 chars
    [String] $hostName
    
    [Net.IpAddress] $ipAddress
    
    Computer([String] $FQDN) {
        if (-not $FQDN.contains('.')) {
            throw "Specify an FQDN e.g. vmName.domain.com"
        }
        
        $this.hostName = $FQDN.Split('.')[0]
        $this.IpAddress = (Test-Connection -ComputerName $FQDN -IPv4 -Count 1).Address.ToString()
    }
    
    [bool] IsAlive() {
        return Test-Connection -ComputerName $this.ipAddress -Count 2 -Quiet
    }
}

In [64]:
$Computer = [Computer]::new("localhost")

Exception: 
Line |
   9 |              throw "Specify an FQDN e.g. vmName.domain.com"
     |              ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Specify an FQDN e.g. vmName.domain.com


In [65]:
$Computer = [Computer]::new($(hostname))

In [66]:
$Computer


hostName   ipAddress
--------   ---------
ddhami-mn2 172.16.37.226



In [67]:
$Computer.IsAlive()


True
